In [15]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot as hvp
import datetime
import altair as alt

In [16]:
# Creating a range of dates
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')
# Dates df
dates_df = pd.DataFrame(date_range, columns=['Date'])
#print(dates_df.head(1825))

# Creating a range of random "prices"
portfolio_df = pd.DataFrame(
    {
    "fake1": np.random.uniform(1000, -500, 100),
    "fake2": np.random.uniform(1000, -500, 100),
    "fake3": np.random.uniform(1000, -500, 100)
    }
    )
f1_portfolio_df = portfolio_df["fake1"].to_frame()

# Concatinating and finalizing test data
returns = pd.concat([dates_df, portfolio_df], join = "inner", axis = 1)
returns.set_index('Date', inplace = True)
returns = pd.DataFrame(returns)
returns.head()

,fake1,fake2,fake3
Date,,,
2018-01-01,841.05,559.57,547.41
2018-01-02,540.89,-7.26,209.09
2018-01-03,-209.58,33.88,-136.60
2018-01-04,-14.14,-286.39,852.62
2018-01-05,466.83,272.84,-282.46


In [17]:
initial_investment = 10000

In [18]:
def cumulative_returns(df):
    return (1 + df).cumprod()

In [19]:
user_cr = cumulative_returns(portfolio_df)
user_cr

,fake1,fake2,fake3
0,842.05,560.57,548.41
1,456295.22,-3507.60,115214.59
2,-95175960.45,-122353.47,-15623428.00
3,1250293337.78,34917864.40,-13336432250.63
4,584928953081.26,9561761715.71,3753693147220.16
...,...,...,...
95,35312096125214629335458687034141881396030180416...,-6867429026005095138796052459523316561429745811...,-2582887973744781219430378244842482405595657013...
96,-7950769813388714786822310290182416728990345768...,15813915555050549818288116143898031913530048088...,-1172729110178211631183771793299779158196105537...
97,26136158328368763521319138797441027849982604505...,-6363357661147628616178459668021609248674883941...,-1547432301691821973087139658257119034456319927...
98,11944221787043471928439677639368192538678799251...,-5428741621888304296641844748130388687017437320...,-1146925766438616922156603092950465635901781814...


## Correlation

In [20]:
returns_cor = returns.corr()
returns_cor

,fake1,fake2,fake3
fake1,1.00,-0.04,0.16
fake2,-0.04,1.00,0.05
fake3,0.16,0.05,1.00


## Correlation Plot

In [21]:
#In the works
# source = returns_cor

# alt.Chart(source).mark_circle(size=60).encode(
#     x='Horsepower',
#     y='Miles_per_Gallon',
#     color='Origin',
#     tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
# ).interactive()

In [22]:
def covariance(df, ticker, market):
    return df[ticker].cov(df[market])

In [23]:
user_covariance = covariance(portfolio_df, 'fake2', 'fake3')
user_covariance

10561.885209700968

In [24]:
def variance(df, market):
    return df[market].var()

In [25]:
user_variance = variance(portfolio_df, 'fake3')
user_variance

180060.51715415236

### Drawdown

In [26]:
def drawdown(df, tickers):
    Roll_Max = df[tickers].cummax()
    Daily_Drawdown = df[tickers]/Roll_Max - 1.0
    Max_Daily_Drawdown = Daily_Drawdown.cummin() * 100

In [27]:
user_drawdown = drawdown(portfolio_df, 'fake1')
user_drawdown

### Drawdown Chart

In [28]:
def drawdown_chart(returns, date, quant):
    '''
    return: is a dataframe
    date: str: should be a date column name
    quant: str: should be a quantitative data type
    '''
    chart = alt.Chart(df.reset_index()).mark_bar().encode(
        x = date + ':T',  # T specifies temporal data type for x-axis
        y = quant + ':Q',  # Q specifies a quantitative data type for y-axis
        color = alt.condition(
            getattr(alt.datum, quant) > 0,
            alt.value("Green"),  # The positive color
            alt.value("red")
    ), tooltip= [date + ':T', quant + ':Q']
    ).properties(width=800)
    chart = chart.interactive()
    return chart

In [29]:
def tracking_error(df, tickers, market):
    track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
    return track_error

In [30]:
user_tracking_error = tracking_error(portfolio_df, 'fake2', 'fake3')
user_tracking_error

6057.6532654634975

In [31]:
def beta(covariance, variance):
    chart_beta = covariance / variance
    chart_beta = chart_beta.rolling(window = 21).plot()
    plt.show()
    beta.shape
    chart_beta.plot()
    return chart_beta

In [32]:
# user_beta = beta(user_covariance, user_variance)
# user_beta
returns.tail()

,fake1,fake2,fake3
Date,,,
2018-04-06,351.77,968.85,847.58
2018-04-07,-23.52,-24.03,453.04
2018-04-08,-329.72,-403.39,130.95
2018-04-09,456.00,852.13,740.18
2018-04-10,-415.59,874.94,468.60


In [39]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

             fake1   fake2   fake3
Date                              
2018-01-01  841.05  559.57  547.41
2018-01-02  540.89   -7.26  209.09
2018-01-03 -209.58   33.88 -136.60
2018-01-04  -14.14 -286.39  852.62
2018-01-05  466.83  272.84 -282.46
...            ...     ...     ...
2018-04-06  351.77  968.85  847.58
2018-04-07  -23.52  -24.03  453.04
2018-04-08 -329.72 -403.39  130.95
2018-04-09  456.00  852.13  740.18
2018-04-10 -415.59  874.94  468.60

[100 rows x 3 columns]


In [34]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
print(user_sharpe_ratio)

fake1   10.36
fake2    8.14
fake3   10.05
dtype: float64


## Return on Investment

In [35]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

## ROI chart

In [36]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

In [42]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,fake1,fake2,fake3
0,841.05,559.57,547.41
1,456294.22,-3508.60,115213.59
2,-95175961.45,-122354.47,-15623429.00
3,1250293336.78,34917863.40,-13336432251.63
4,584928953080.26,9561761714.71,3753693147219.16
...,...,...,...
95,35312096125214629335458687034141881396030180416...,-6867429026005095138796052459523316561429745811...,-2582887973744781219430378244842482405595657013...
96,-7950769813388714786822310290182416728990345768...,15813915555050549818288116143898031913530048088...,-1172729110178211489983073857888904310740663427...
97,26136158328368766482512399563868897930891357829...,-6363357661147627839919413517667141674185139710...,-1547432301691821973087139658257119034456319927...
98,11944221787043471928439677639368192538678799251...,-5428741621888304296641844748130388687017437320...,-1146925766438616922156603092950465635901781814...


In [ ]:
source = data.wheat()

bar = alt.Chart(source).mark_bar().encode(
    x = df + ':O',
    y = returns + ':Q'
)

<!-- line = alt.Chart(source).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(wheat)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='year:O',
    y='rolling_mean:Q'
)

(bar + line).properties(width=600) -->

   ## Porfolio Size and Holdings

In [38]:
# In the works
# source = returns
def portfolio_pie (portfolio_df)
# alt.Chart(source).mark_arc().encode(
#     theta="value",
#     color = returns
# )